# Topic Modeling using LDA

In [1]:
# # To be run only once
# if 0 == 1:
#     !pip install gensim
#     !pip install PyLDAvis
#     !pip install spacy
#     !python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt

import re
import spacy
import tqdm

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
import en_core_web_sm

from pprint import pprint

import pyLDAvis.gensim
import pickle 
import pyLDAvis

import time
from collections import Counter
import random

In [3]:
random.seed(3)

In [4]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asaid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Preprocessing data

In [6]:
start = time.time()

In [7]:
# one_drive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"
one_drive_path = "C:/Users/asaid/The Estée Lauder Companies Inc/TeamAnis - General/"

In [8]:
# reviews = pd.read_pickle('reviews.pickle')

# cols = ['type', 'onlinepost_id', 'source_product_identifier', 'onlinestatement_id',
#        'date', 'title', 'description', 'geography', 'channel', 'product_id',
#        'rating', 'sentiment']
# reviews = reviews[cols]

# reviews.loc[reviews['type']=='Cosmetics','type']='temps'
# reviews.loc[reviews['type']=='Skincare','type']='Cosmetics'
# reviews.loc[reviews['type']=='temps','type']='Skincare'

# reviews = reviews[reviews['geography']=='USA']
# reviews.drop(columns=['geography'], inplace=True)

# reviews.to_pickle('reviews_filtered.pickle')

In [9]:
# reviews = pd.read_pickle('reviews_filtered.pickle')

In [10]:
# # Creating date columns in the right dtype and dropping the day of the month: 2019-02-24 => 2019-02-01
# reviews.loc[:,'clean_date'] = pd.to_datetime(reviews['date'], errors='coerce')
# if reviews['clean_date'].isna().sum() > 0:
#     print('{} rows have been dropped because the date format is wrong.'.format(reviews['clean_date'].isna().sum()))
#     display(reviews.loc[reviews['clean_date'].isna(), 'date'])
#     reviews = reviews.dropna(subset='date')
# reviews['date'] = reviews['clean_date']  
# reviews = reviews.drop('clean_date', axis=1)
# reviews['date'] = reviews['date'].dt.to_period('m')

# # Checking for missing data (NA => -1)
# if reviews['rating'].isna().sum()>0:
#     print('{} rows are missing ratings'.format(reviews['rating'].isna().sum()))
#     reviews.loc[:,'rating'] = reviews['rating'].fillna(-1).astype(int)

# if reviews['sentiment'].isna().sum()>0:
#     print('{} rows are missing sentiments'.format(reviews['sentiment'].isna().sum()))
#     reviews.loc[:,'sentiment'] = reviews['sentiment'].fillna(-1).astype(int)
    
# # Transforming rating and sentiment to dummy variables (one-hot encoding)
# reviews.loc[:,'sentiment'] = reviews['sentiment'].str.lower()
# reviews.loc[:,'rating'] = reviews['rating'].astype(int)
# reviews = pd.concat([reviews, pd.get_dummies(data=reviews[['rating','sentiment']], columns=['rating','sentiment'], dtype=int)], axis=1)

# # Readding NAs data to ratings
# reviews.loc[reviews['rating']==-1,'rating'] = np.nan
# reviews.loc[reviews['sentiment']==-1,'sentiment'] = np.nan

# # Transforming sentiment to integer data (positive:1; netural:0, negative:-1)
# reviews.loc[:,'sentiment'] = reviews['sentiment_positive'] - reviews['sentiment_negative']

# # Aggregating RR data by OnlinePost_ID

# # Creating a column to count the number of statements by review once aggreagtion happens
# reviews['nb_statements'] = reviews['sentiment']

# reviews = reviews.groupby(['type',
#                            'channel',
#                            'source_product_identifier',  
#                            'date',
#                            'onlinepost_id']).agg({
#     'description': lambda x:'. '.join(list(x)),
#     'nb_statements':'count',
#     'rating':'first',
#     'rating_1':'first',
#     'rating_2':'first',
#     'rating_3':'first',
#     'rating_4':'first',
#     'rating_5':'first',
#     'sentiment_negative':'sum',
#     'sentiment_neutral':'sum',
#     'sentiment_positive':'sum',
#     'sentiment':'mean'
# }).reset_index()

# # Normalize the one hot sentiment encoding counts (sentiment_negative, sentiment_neutral, sentiment_positive) by the nb_statement.
# reviews[['sentiment_negative', 'sentiment_neutral', 'sentiment_positive']] = reviews[['sentiment_negative', 'sentiment_neutral', 'sentiment_positive']].div(reviews['nb_statements'], axis=0)

# reviews.to_pickle('reviews_wrangled.pickle')

In [11]:
reviews = pd.read_pickle('reviews_wrangled.pickle')

In [12]:
# Adding product related words to the stop words
stop_words = stopwords.words('english')
stop_words.extend(['from'])

brands = pd.read_csv('elc_brands.csv', encoding='ISO-8859-1')
catalogue = pd.read_csv('elc_catalogue.csv', encoding='ISO-8859-1')

stop_words.extend(list(set(str(w).lower() for w in ' '.join(brands['ELC_Brand'].unique().tolist() +
                                          catalogue['Major_Category'].unique().tolist() +
                                          catalogue['Application'].unique().tolist() +
                                          catalogue['Category'].unique().tolist() +
                                          catalogue['SubCategory'].unique().tolist() 
                                         ).replace('/', ' ').split())))

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
stop_words_nlp = nlp(' '.join([' '.join(gensim.utils.simple_preprocess(str(word), deacc=True)) for word in stop_words]))
stop_words = [token.lemma_ for token in stop_words_nlp]


# Cache stop_words into hash
stop_words = Counter(stop_words)


def preprocess(sentences, stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    for sentence in sentences:
        doc = nlp(' '.join([token for token in gensim.utils.simple_preprocess(str(sentence), deacc=True)]) )
        yield([token.lemma_ for token in doc if (token.pos_ in allowed_postags) and (not (token.lemma_ in stop_words))])

In [13]:
'lipstick' in stop_words

True

In [14]:
reviews['tokens'] = list(tqdm.tqdm(preprocess(reviews['description'].values.tolist(), stop_words), position=0, leave=True, total=len(reviews)))

100%|█████████████████████████████████████████████████████████████████████| 4636608/4636608 [4:21:43<00:00, 295.26it/s]


In [15]:
reviews.to_pickle('reviews_wrangled_w_tokens.pickle')

In [16]:
reviews = pd.read_pickle('reviews_wrangled_w_tokens.pickle')

In [17]:
rp = pd.read_csv('reviews_products.csv', low_memory=False)
products = rp[['type', 'channel', 'source_product_identifier', 'product', 'brand_abbrev', 'elc_brand',
       'brand_score', 'item_description', 'product_score', 'itemid_4',
       'major_category_id', 'major_category', 'application_id', 'application',
       'category_id', 'category', 'sub_category_id', 'sub_category']].drop_duplicates()
products = products.dropna(subset=['elc_brand'], axis=0)

In [18]:
reviews  = reviews.merge(products, left_on=['type', 'channel', 'source_product_identifier'], right_on=['type', 'channel', 'source_product_identifier'], how='inner')

In [19]:
reviews.to_pickle('reviews_products.pickle')

In [20]:
# reviews = pd.read_pickle('reviews_products.pickle')

In [21]:
reviews

,type,channel,source_product_identifier,date,onlinepost_id,description,nb_statements,rating,rating_1,rating_2,rating_3,rating_4,rating_5,sentiment_negative,sentiment_neutral,sentiment_positive,sentiment,tokens,product,brand_abbrev,elc_brand,brand_score,item_description,product_score,itemid_4,major_category_id,major_category,application_id,application,category_id,category,sub_category_id,sub_category
0,Skincare,Amazon USA,B00015GYC2,2015-03,OnlinePost_20200301_24654a95-b252-4dee-8dfa-77...,This was very disappointing.. When I received ...,4,1.0,1,0,0,0,0,0.5,0.500000,0.000000,-0.500000,"[disappointing, receive, cleansing, use, night...",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
1,Skincare,Amazon USA,B00015GYC2,2017-01,OnlinePost_20200301_84729e90-665d-4108-8ca5-0b...,I have tried many face washes- this is the bes...,2,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[try, many, good, get, dirt, soft]",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
2,Skincare,Amazon USA,B00015GYC2,2017-12,OnlinePost_20200301_9c216502-fba9-48a8-95cb-3b...,Fast service.. could find it from my usual ven...,3,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[fast, could, find, usual, vendor, one, fast, ...",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
3,Skincare,Amazon USA,B00015GYC2,2018-01,OnlinePost_20200301_01d7a936-32dd-454d-b4ef-ab...,Five Stars.. great great product.,2,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[star, great, great]",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
4,Skincare,Amazon USA,B00015GYC2,2018-07,OnlinePost_20200301_53a09274-200c-47dc-9cca-52...,Order from La Mer only.. Can't go wrong with L...,3,4.0,0,0,0,1,0,0.0,0.666667,0.333333,0.333333,"[order, go, wrong, order, site]",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388052,Cosmetics,Ulta,xlsImpprod6490080,2019-06,OnlinePost_20191027_184332048,Maybe I got an old one but this thing was awfu...,6,1.0,1,0,0,0,0,0.5,0.166667,0.333333,-0.166667,"[maybe, get, old, thing, awful, bad, batch, pa...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup Prime
388053,Cosmetics,Ulta,xlsImpprod6490080,2019-06,OnlinePost_20191027_185330102,Love most Smashbox products and used the origi...,5,1.0,1,0,0,0,0,0.2,0.400000,0.400000,0.200000,"[love, use, original, year, disappoint, disapp...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup Prime
388054,Cosmetics,Ulta,xlsImpprod6490080,2019-07,OnlinePost_20191027_184010998,The Best Primer.. this is literally the best p...,3,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[good, literally, good, ever, use, use, else, ...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup Prime
388055,Cosmetics,Ulta,xlsImpprod6490080,2019-07,OnlinePost_20191027_184176048,I didn't wear it for 24 hours but my eyeshadow...,2,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[wear, hour, crease, stay, entire, time, exact...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup Prime


## Using class

In [22]:
class TopicModeller():
    def __init__(self, df, column):
        self.df = df.reset_index(drop=True)
        self.column = column
        self.docs = df[column].values.tolist()
    
    def compute_coherence_values(self, corpus, id2word, text, k, a, b):

        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=k, 
                                               random_state=3,
                                               chunksize=100,
                                               passes=10,
                                               alpha=a,
                                               eta=b)

        coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')

        return coherence_model_lda.get_coherence()
    
    def train_valid_lda(self,
                        nb_samples: int = 100000,
                        topics_range: list = [3,5,7,9],
                        alpha: list = [0.1, 0.01, 'symmetric','asymmetric'],
                        beta: list = [0.1, 'symmetric']                        
                        ):
        
        nb_samples = min(nb_samples, len(self.docs))
        
        # Training set
        train = random.sample(self.docs, nb_samples)
        
        # Create Dictionary
        self.id2word = corpora.Dictionary(train)
        
        # Term Document Frequency
        self.corpus = [self.id2word.doc2bow(text) for text in train]
        
        # Result dictionary
        self.model_results = {'Topics': [],
                         'Alpha': [],
                         'Beta': [],
                         'Coherence': []
                        }

        # Can take a long time to run
        print('Validating hyperparameters...')
        if 1 == 1:
            pbar = tqdm.tqdm(total=(len(beta)*(len(alpha))*len(topics_range)), position=0, leave=True)
            # iterate through number of topics
            for k in topics_range:
                # iterate through alpha values
                for a in alpha:
                    # iterare through beta values
                    for b in beta:
                        # get the coherence score for the given parameters
                        cv = self.compute_coherence_values(corpus=self.corpus, id2word=self.id2word, text=train,
                                                      k=k, a=a, b=b)
                        # Save the model results
                        self.model_results['Topics'].append(k)
                        self.model_results['Alpha'].append(a)
                        self.model_results['Beta'].append(b)
                        self.model_results['Coherence'].append(cv)
                        print('nb topics: {}, alpha: {}, beta: {}, coherence: {}'.format(k,a,b,cv))
                        pbar.update(1)
        self.cv_results = pd.DataFrame(self.model_results).sort_values("Coherence", ascending=False)
        self.cv_results.to_csv('lda_tuning_results.csv', index=False)
        pbar.close()
        
        self.best_param = self.cv_results.iloc[0]
        self.best_num_topics = int(self.best_param['Topics'])
        self.best_alpha = self.best_param['Alpha']
        self.best_beta = self.best_param['Beta']
        name = 'cv_best'
        
        # Build LDA model
        print('Training best model...')
        self.lda_model = gensim.models.LdaMulticore(corpus=self.corpus,
                                                   id2word=self.id2word,
                                                   num_topics=self.best_num_topics,
                                                   alpha = self.best_alpha,
                                                   eta = self.best_beta,
                                                   random_state=3,
                                                   chunksize=100,
                                                   passes=10,
                                                   per_word_topics=True)
        print('Done.')
        pyLDAvis.enable_notebook()
        LDAvis_prepared = pyLDAvis.gensim.prepare(self.lda_model, self.corpus, self.id2word)
        display(LDAvis_prepared)
        pyLDAvis.save_html(LDAvis_prepared, name+'_'+str(self.best_alpha)+'_'+str(self.best_beta)+'_'+str(self.best_num_topics)+'.html')
        self.lda_model.save(name+'_'+str(self.best_alpha)+'_'+str(self.best_beta)+'_'+str(self.best_num_topics))
        pickle.dump(self.id2word, open( "id2word_"+name+'_'+str(self.best_alpha)+'_'+str(self.best_beta)+'_'+str(self.best_num_topics)+".pickle", "wb" ))
        return self.lda_model, self.id2word
        
    def get_docs_topics(self):
        # Term Document Frequency
        corpus = [self.id2word.doc2bow(text) for text in self.docs]
        self.output = pd.concat([self.df, pd.DataFrame(gensim.matutils.corpus2csc(self.lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,self.best_num_topics+1)])], axis=1)
        return self.output

    def view(self):
        pyLDAvis.enable_notebook()
        self.LDAvis_prepared = pyLDAvis.gensim.prepare(self.lda_model, self.corpus, self.id2word)
        display(LDAvis_prepared)

In [23]:
all_lda = TopicModeller(reviews, 'tokens')
lda, id2word = all_lda.train_valid_lda()

  0%|                                                                                           | 0/32 [00:00<?, ?it/s]

Validating hyperparameters...


  3%|██▌                                                                             | 1/32 [03:21<1:44:19, 201.92s/it]

nb topics: 3, alpha: 0.1, beta: 0.1, coherence: 0.43450483502760867


  6%|█████                                                                           | 2/32 [06:59<1:43:18, 206.61s/it]

...

nb topics: 9, alpha: asymmetric, beta: 0.1, coherence: 0.5376250455256808


100%|███████████████████████████████████████████████████████████████████████████████| 32/32 [1:55:26<00:00, 216.45s/it]

nb topics: 9, alpha: asymmetric, beta: symmetric, coherence: 0.5188056235047867
Training best model...


Done.


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.066201  0.118931       1        1  20.215582
0      0.136360  0.051055       2        1  18.425123
3      0.024137  0.137291       3        1  15.321939
2      0.140470  0.131328       4        1  12.971735
7     -0.088396  0.034800       5        1   8.255013
6      0.179293 -0.157810       6        1   6.515980
5     -0.007944 -0.353759       7        1   6.341194
8      0.066495  0.049640       8        1   6.129386
4     -0.384214 -0.011476       9        1   5.824048, topic_info=         Term          Freq         Total Category  logprob  loglift
6        love  47765.000000  47765.000000  Default  30.0000  30.0000
124  coverage  14228.000000  14228.000000  Default  29.0000  29.0000
25      shade   9271.000000   9271.000000  Default  28.0000  28.0000
43       long  15115.000000  15115.000000  Default  27.0000  27.0000
90       oily   9010.000000   9010.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
46       time   1088.588501  10269.834961   Topic9  -4.3967   0.5988
293       see    738.170349   3622.906250   Topic9  -4.7851   1.2523
80        new    733.587158   4315.590820   Topic9  -4.7913   1.0711
102      come    700.762451   5340.246094   Topic9  -4.8371   0.8123
88      would    653.378662  11844.045898   Topic9  -4.9071  -0.0542

[574 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
583       3  0.998049    absolute
438       2  0.830248  absolutely
438       3  0.169579  absolutely
167       1  0.987745      absorb
167       5  0.011203      absorb
...     ...       ...         ...
1651      5  0.998379       write
48        3  0.998023        year
48        6  0.001925        year
314       7  0.998891      yellow
437       8  0.998549        zone

[965 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3, 8, 7, 6, 9, 5])

In [24]:
def train_display_save(tokens, nb_samples, num_topics, alpha, eta, name):
    nb_samples = min(nb_samples, len(tokens))
        
    # Training set
    train = tokens.sample(nb_samples, random_state=3).values.tolist()

    # Create Dictionary
    id2word = corpora.Dictionary(train)

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in train]
    start=time.time()
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                                   id2word=id2word,
                                                   num_topics=num_topics,
                                                   alpha =alpha ,
                                                   eta =eta ,
                                                   random_state=3,
                                                   chunksize=100,
                                                   passes=10,
                                                   per_word_topics=True)
    pyLDAvis.enable_notebook()
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    display(LDAvis_prepared)
    pyLDAvis.save_html(LDAvis_prepared, name+'_'+str(alpha)+'_'+str(eta)+'_'+str(num_topics)+'.html')
    lda_model.save(name+'_'+str(alpha)+'_'+str(eta)+'_'+str(num_topics))
    pickle.dump(id2word, open( "id2word_"+ name+'_'+str(alpha)+'_'+str(eta)+'_'+str(num_topics)+".p", "wb" ))
    print(time.time()-start)
    return lda_model, id2word

In [25]:
# lda, id2word = train_display_save(reviews['tokens'], 500000, 8, alpha='asymmetric', eta='symmetric', name='full')

In [26]:
docs = reviews['tokens'].values.tolist()

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

topics = gensim.matutils.corpus2csc(lda.get_document_topics(corpus)).T.toarray()

topics = pd.DataFrame(topics, columns=['topic_'+str(i) for i in range(1,topics.shape[1]+1)])

output = pd.concat([reviews, topics], axis=1)

output.to_pickle('reviews_w_topics.pkl')

In [80]:
topics = lda.print_topics(num_words=6)
for topic in topics:
    print('Topic #{}'.format(topic[0]+1))
    print('Terms: ', topic[1])

Topic #1
Terms:  0.033*"buy" + 0.031*"product" + 0.022*"so" + 0.022*"will" + 0.021*"use" + 0.020*"try"
Topic #2
Terms:  0.041*"love" + 0.041*"sample" + 0.034*"receive" + 0.033*"size" + 0.029*"all" + 0.027*"shadow"
Topic #3
Terms:  0.056*"skin" + 0.036*"primer" + 0.028*"product" + 0.027*"make" + 0.026*"feel" + 0.026*"use"
Topic #4
Terms:  0.038*"shade" + 0.034*"color" + 0.029*"too" + 0.025*"skin" + 0.025*"light" + 0.017*"more"
Topic #5
Terms:  0.069*"use" + 0.049*"good" + 0.048*"love" + 0.047*"year" + 0.044*"product" + 0.034*"ever"
Topic #6
Terms:  0.071*"product" + 0.054*"great" + 0.045*"eye" + 0.035*"star" + 0.034*"use" + 0.031*"concealer"
Topic #7
Terms:  0.064*"foundation" + 0.037*"coverage" + 0.029*"skin" + 0.028*"look" + 0.025*"day" + 0.020*"powder"
Topic #8
Terms:  0.066*"mascara" + 0.038*"lash" + 0.035*"eye" + 0.021*"eyeliner" + 0.021*"look" + 0.019*"love"
Topic #9
Terms:  0.097*"color" + 0.051*"lip" + 0.048*"love" + 0.036*"lipstick" + 0.022*"great" + 0.020*"look"


In [ ]:
train_display_save(reviews['tokens'], 100000, 10, alpha='asymmetric', eta='symmetric', name='full')

In [ ]:
train_display_save(reviews['tokens'], 100000, 9, alpha='asymmetric', eta='symmetric', name='full')

In [ ]:
train_display_save(reviews['tokens'], 100000, 8, alpha='asymmetric', eta='symmetric', name='full')

In [31]:
train_display_save(reviews['tokens'], 100000, 7, alpha='symmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.118104 -0.010693       1        1  20.290110
2      0.014560 -0.145870       2        1  19.437847
1     -0.065970 -0.097884       3        1  19.360294
3     -0.065640 -0.201709       4        1  13.120198
0      0.223173  0.040887       5        1  10.240201
4      0.124811  0.252209       6        1   9.819157
6     -0.349038  0.163060       7        1   7.732195, topic_info=          Term          Freq         Total Category  logprob  loglift
26        love  45834.000000  45834.000000  Default  30.0000  30.0000
37        good  27886.000000  27886.000000  Default  29.0000  29.0000
218      shade   9027.000000   9027.000000  Default  28.0000  28.0000
100  recommend   7596.000000   7596.000000  Default  27.0000  27.0000
28    coverage  14184.000000  14184.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
15        look   2018.469604  30175.013672   Topic7  -4.0624  -0.1449
159      blend   1153.179688   4936.708008   Topic7  -4.6223   1.1056
18        nice   1302.536133   8656.076172   Topic7  -4.5005   0.6658
529      matte    917.440491   2909.153809   Topic7  -4.8510   1.4057
19    purchase    916.242859   7559.958496   Topic7  -4.8523   0.4494

[470 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
157       3  0.999185    absolute
334       1  0.064053  absolutely
334       3  0.808745  absolutely
334       5  0.125798  absolutely
334       7  0.001443  absolutely
...     ...       ...         ...
879       2  0.998159     wrinkle
546       1  0.998304       write
24        3  0.372624        year
24        6  0.627320        year
779       7  0.999036      yellow

[763 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 2, 4, 1, 5, 7])

254.2011616230011


(<gensim.models.ldamulticore.LdaMulticore at 0x17c2abf5a08>,
 <gensim.corpora.dictionary.Dictionary at 0x17d460b8948>)

In [ ]:
train_display_save(reviews['tokens'], 100000, 6, alpha=0.1, eta='symmetric', name='full')

In [ ]:
train_display_save(reviews['tokens'], 100000, 5, alpha='asymmetric', eta=0.1, name='full')

In [ ]:
train_display_save(reviews['tokens'], 100000, 4, alpha='asymmetric', eta=0.1, name='full')

In [ ]:
train_display_save(reviews['tokens'], 100000, 3, alpha='asymmetric', eta=0.1, name='full')

In [ ]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

## Splitting by review

In [ ]:
bad_reviews['tokens'].sample(100000)

In [ ]:
good_reviews_6 = train_display_save(tokens=good_reviews['tokens'], nb_samples=100000, num_topics=6, alpha='asymmetric', eta=0.1, name='good_reviews')

In [ ]:
bad_reviews_6 = train_display_save(tokens=bad_reviews['tokens'], nb_samples=100000, num_topics=6, alpha='asymmetric', eta=0.1, name='bad_reviews')

In [ ]:
neutral_reviews_6 = train_display_save(tokens=neutral_reviews['tokens'], nb_samples=100000, num_topics=6, alpha='asymmetric', eta=0.1, name='neutral_reviews')

In [ ]:
def save_display(self, title):
    p = pyLDAvis.gensim.prepare(self.lda_model, self.corpus, self.id2word)
    pyLDAvis.save_html(p, title+'.html')

In [ ]:
save_display(good_lda, 'good_reviews_8_topics')

In [ ]:
save_display(bad_lda, 'bad_reviews_8_topics')

In [ ]:
save_display(neutral_lda, 'neutral_reviews_8_topics')

In [ ]:
good_reviews = reviews[reviews['rating']>=4]
neutral_reviews = reviews[(reviews['rating']>2) & (reviews['rating']<4)]
bad_reviews = reviews[reviews['rating']<=2]

In [ ]:
print(len(good_reviews))
good_lda = TopicModeller(good_reviews, 'tokens')
good_lda.train_valid_lda()

In [ ]:
display(good_lda.cv_results)

In [ ]:
good_lda.view()

In [ ]:
print(len(bad_reviews))
bad_lda = TopicModeller(bad_reviews, 'tokens')
bad_lda.train_valid_lda()
bad_lda.view()

In [ ]:
display(bad_lda.cv_results)

In [ ]:
bad_lda.view()

In [ ]:
print(len(neutral_reviews))
neutral_lda = TopicModeller(neutral_reviews, 'tokens')
neutral_lda.train_valid_lda()
neutral_lda.view()

In [ ]:
display(neutral_lda.cv_results)

In [ ]:
neutral_lda.view()

In [ ]:
good_reviews = good_lda.get_docs_topics()

In [ ]:
neutral_reviews = neutral_lda.get_docs_topics()

In [ ]:
bad_reviews = bad_lda.get_docs_topics()

In [ ]:
good_reviews.to_pickle('good_reviews.pickle')
bad_reviews.to_pickle('bad_reviews.pickle')
neutral_reviews.to_pickle('neutral_reviews.pickle')

In [ ]:
good_lda.cv_results

In [ ]:
good_reviews_3 = train_display_save(tokens=good_reviews['tokens'], nb_samples=100000, num_topics=3, alpha='asymmetric', eta=0.1, name='good_reviews')
bad_reviews_3 = train_display_save(tokens=bad_reviews['tokens'], nb_samples=100000, num_topics=3, alpha='asymmetric', eta=0.1, name='bad_reviews')
neutral_reviews_3 = train_display_save(tokens=neutral_reviews['tokens'], nb_samples=100000, num_topics=3, alpha='asymmetric', eta='symmetric', name='neutral_reviews')

In [ ]:
int_start=time.time()
# Inputs
tokens_good = random.sample(good_reviews['tokens'].values.tolist(),100000)
# Create Dictionary
id2word_good = corpora.Dictionary(tokens_good)
# Term Document Frequency
corpus_good = [id2word_good.doc2bow(text) for text in tokens_good]
print(time.time()-int_start)

# Build LDA model
int_start=time.time()
lda_model_good = gensim.models.LdaMulticore(corpus=corpus_good,
                                       id2word=id2word_good,
                                       num_topics=3,
                                       alpha = 'asymmetric',
                                       eta = 0.1,
                                       random_state=3,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_good, corpus_good, id2word_good)
LDAvis_prepared

In [ ]:
pyLDAvis.gensim.prepare(lda_model_good, corpus_good, id2word_good)

In [ ]:
bad_lda.cv_results

In [ ]:
int_start=time.time()
# Inputs
tokens_bad = random.sample(bad_reviews['tokens'].values.tolist(),100000)
# Create Dictionary
id2word_bad = corpora.Dictionary(tokens_bad)
# Term Document Frequency
corpus_bad = [id2word_bad.doc2bow(text) for text in tokens_bad]
print(time.time()-int_start)

# Build LDA model
int_start=time.time()
lda_model_bad = gensim.models.LdaMulticore(corpus=corpus_bad,
                                       id2word=id2word_bad,
                                       num_topics=3,
                                       alpha = 'asymmetric',
                                       eta = 0.1,
                                       random_state=3,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_bad, corpus_bad, id2word_bad)
LDAvis_prepared

In [ ]:
int_start=time.time()
# Inputs
tokens_neutral = random.sample(neutral_reviews['tokens'].values.tolist(),100000)
# Create Dictionary
id2word_neutral = corpora.Dictionary(tokens_neutral)
# Term Document Frequency
corpus_neutral = [id2word_neutral.doc2bow(text) for text in tokens_neutral]
print(time.time()-int_start)

# Build LDA model
int_start=time.time()
lda_model_neutral = gensim.models.LdaMulticore(corpus=corpus_neutral,
                                       id2word=id2word_neutral,
                                       num_topics=3,
                                       alpha = 'asymmetric',
                                       eta = 'symmetric',
                                       random_state=3,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_neutral, corpus_neutral, id2word_neutral)
LDAvis_prepared

## NLP Preprocessing

In [ ]:
docs = pickle.load(open("docs.pickle", "rb"))

d = corpora.Dictionary(docs)

freq = pd.DataFrame(d.dfs.values(), index=d.dfs.keys(), columns=['freq'])
freq.index.name = 'idx'
freq = freq.reset_index()
freq['token'] = freq['idx'].apply(lambda x:d[x])
freq = freq.sort_values(by='freq', ascending=False)

In [ ]:
freq[freq['freq']>500]

## Validating Model

In [ ]:
# Inputs
docs = random.sample(docs, 100000)

# Create Dictionary
id2word = corpora.Dictionary(docs)
pickle.dump(id2word, open( "id2word.p", "wb" ) )

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, text, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
grid = {}
grid['Validation_Set'] = {}

nb_words = len(id2word)

# Topics range
min_topics = 6
max_topics = 10
step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
topics_range = [8]
# Alpha parameter
# Added in the loop

# Alpha
alpha = [
#     0.1, 
#     'symmetric',
#     'asymmetric'
]

# Beta parameter
beta = [
    0.1, 
#     200/nb_words
]
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.05), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
    self.corpus
]

corpus_title = [
#     '25% Corpus',
#     '50% Corpus',
#     '75% Corpus',
    '100% Corpus'
]

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*(len(alpha)+1)*len(topics_range)))
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            alpha.append(50/k)
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, text=docs,
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    pbar.update(1)
    res = pd.DataFrame(model_results)
    res = pd.DataFrame(model_results).sort_values("Coherence", ascending=False)
    res.to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

In [ ]:
res

## Training Best Model

In [ ]:
best_param = res.iloc[0]
num_topics = best_param['Topics']
alpha = best_param['Alpha']
eta = best_param['Beta']

In [ ]:
# Build LDA model
int_start=time.time()
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       alpha = alpha,
                                       eta = eta,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

In [ ]:
lda_model.save('lda_test.model')

In [ ]:
print(time.time()-start)

## Predictions

In [ ]:
# df = pd.read_pickle('reviews_concat.pkl')

In [ ]:
# df = df.loc['2019']

In [ ]:
# print(len(df))

In [ ]:
# docs = list(tqdm.tqdm(preprocess(df.values.tolist(), stop_words), position=0, leave=True))

In [ ]:
# pickle.dump(docs, open( "docs.p", "wb" ) )

In [ ]:
# docs = pickle.load(open("docs.p", "rb"))
# id2word = pickle.load(open("id2word.p", "rb"))

# # Term Document Frequency
# corpus = [id2word.doc2bow(text) for text in docs]

# output = pd.concat([df.to_frame('description').reset_index(), pd.DataFrame(gensim.matutils.corpus2csc(lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,num_topics+1)])], axis=1, ignore_index=True)

# output.to_pickle('reviews_w_topics_test.pkl')

## Appendix

In [ ]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=150) # higher threshold fewer phrases.
# # trigram = gensim.models.Phrases(bigram[data_words], threshold=150)

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# # trigram_mod = gensim.models.phrases.Phraser(trigram)

# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

# # Remove Stop Words
# data_words = remove_stopwords(data_words)

# # Do lemmatization keeping only noun, adj, vb, adv
# data_words = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# # Form Bigrams
# data_words = make_bigrams(data_words)

In [ ]:
# # Build LDA model
# lda_model = gensim.models.LdaMulticore(corpus=corpus,
#                                        id2word=id2word,
#                                        num_topics=8, 
#                                        random_state=100,
#                                        chunksize=100,
#                                        passes=10,
#                                        per_word_topics=True)

In [ ]:
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [ ]:
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('Coherence Score: ', coherence_lda)